## target
task : img classification

data : CIFAR10

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

import numpy as np

In [ ]:
# Hyper parameters
EPOCH = 1
BATCH_SIZE = 100
LR = 0.01
DOWNLOAD_CIFAR10 = False # 請求下載

In [ ]:
### load cifar image

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True,
                                        download=DOWNLOAD_CIFAR10, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False,
                                       download=DOWNLOAD_CIFAR10, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
print('trainset traindata type', type(trainset.train_data))
print('trainset traindata shape', trainset.train_data.shape)
print('------------------')
print('trainset trainlabel type', type(trainset.train_labels))
print('trainset trainlabel shape', len(trainset.train_labels))

In [ ]:
print('testset testdata type', type(testset.test_data))
print('testset testdata shape', testset.test_data.shape)
print('------------------')
print('testset testdata type', type(testset.test_labels))
print('testset testdata shape', len(testset.test_labels))

In [ ]:
### show some img data, only work when trainloader has been declare

import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))
plt.show()

## build CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self, n_conv1_filter, n_conv2_filter):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(# input size(3, 32, 32)
            nn.Conv2d( 
                in_channels = 3, # RGB
                out_channels = n_conv1_filter, # num_filter
                kernel_size=5, # filter's height and width = 5
                stride=1, # shift 1 distance each time
                padding=2, # if stride=1, padding=(kernel_size-1)/2, filled with zero
            ),# -->(16, 32, 32)
            nn.ReLU(),# -->(16, 32, 32)
            nn.MaxPool2d(kernel_size=2,),# -->(16, 16, 16)
        )
        self.conv2 = nn.Sequential(# input size(16, 16, 16)
            nn.Conv2d(n_conv1_filter, n_conv2_filter, 5, 1, 2), #(in, out, kernel, strid, padding) # -->(32, 16, 16)
            nn.ReLU(),# -->(32, 16, 16)
            nn.MaxPool2d(2), # -->(32, 8, 8)
        )
        self.out = nn.Linear(n_conv2_filter * 8 * 8 , 10) # 10 class
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x) # (batch, 32, 7, 7)
        x = x.view(x.size(0), -1) #(batch, 32*7*7)
        output = self.out(x)
        return output

In [ ]:
result = {}

result['param_num'] = []
result['train_acc'] = []
result['test_acc'] = []
result['train_loss'] = []
result['test_loss'] = []

In [ ]:
# training and validate 20 different model

for itr in range(1, 21, 1):

    num_filter = itr * 2 + 4
    
    # declare CNN model with num_filter
    cnn=CNN(n_conv1_filter = num_filter, n_conv2_filter = (num_filter*2))
#     print(cnn)

    # count parameters number
    param_num = 0

    for parameter in cnn.parameters(): # find each layer's parameter tensor
        param_each_layer = 1
        for dim in list(parameter.size()): # calculate parameter number
            param_each_layer *=dim

        param_num += param_each_layer # sum

    print('itr %d | param_num %d'% (itr, param_num)) # finally get number of parameters
    
    def training():
        optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
        loss_func = nn.CrossEntropyLoss()

        for epoch in range(EPOCH):
            for step, (x, y) in enumerate(trainloader):
#                 print(type(x),' | ' , type(y))
                b_x = Variable(x, requires_grad=True) # batch x
                b_y = Variable(y) # batch y

                output = cnn(b_x)
                loss = loss_func(output, b_y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                break
                
#########################################################                        
    
        ### train
#         train_loss
        train_loss = loss.data[0]
        # train_acc
        correct = 0
        total = 0
        
        train_x = None
        train_y = None
        
        for itr, data in enumerate(trainloader):
            images, labels = data
            if itr==0:
                train_x = images
                train_y = labels
            else:
                train_x = torch.cat((train_x, images), dim=0)
                train_y = torch.cat((train_y, labels), dim=0)
        print('train_x shape', train_x.size(), ' | train_y shape', train_y.size())
        outputs = cnn(Variable(train_x))
        _, predicted = torch.max(outputs.data, 1)
        total = train_x.size(0)
        correct = (predicted == labels).sum()
        train_acc = correct / total
        
        ### test
#         test_x = Variable(torch.from_numpy(testset.train))
#         print(test_x.shape)
#         test_pred = cnn.forward(test_x)
#         # test_loss
#         test_loss = loss_func(test_pred, Variable(test_y)).data[0]
#         # test_acc

#         correct = 0
#         total = 0

#         for i, data in enumerate(testloader4eval):
#             images, labels = data
#             print(i, ' | ', images.shape)
#             outputs = cnn(Variable(images))
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum()
#         test_acc = correct / total
    
#########################################################        

        ### record
        result['param_num'].append(round(param_num, 4))
        result['train_acc'].append(round(train_acc, 4))
#         result['test_acc'].append(round(test_acc, 4))
        result['train_loss'].append(round(train_loss, 4))
#         result['test_loss'].append(round(test_loss, 4))
        
        ### print
        print(
            'train_acc : %.4f' % train_acc,
#             '\t test_acc : %.4f' % test_acc,
            '\t train_loss : %.4f' % train_loss,
#             '\t test_loss : %.4f' % test_loss,
        )
        
#         del cnn
        
    training()

In [ ]:
a = torch.FloatTensor([[1,2,3], [4,5,5]])
print(a.size())
print(a.shape)

In [ ]:
trainloader.